In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Tabela 8.2: Estimação MPQ da equação nettfa
             (WLS Estimation of the nettfa Equation)
             
Arquivo com os dados: 401ksubs.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

In [2]:
df = pd.read_excel('401ksubs.xls',
                   header=None,
                   usecols=[0, 1, 3, 4, 5, 6],
                   names=['e401k', 'inc', 'male', 'age', 'fsize', 'nettfa'])

In [3]:
df.head()

,e401k,inc,male,age,fsize,nettfa
0,0,13.170,0,40,1,4.575
1,1,61.230,1,35,1,154.000
2,0,12.858,0,44,2,0.000
3,0,98.880,1,44,2,21.800
4,0,22.614,0,53,1,18.450


In [4]:
filtro = (df['fsize'] == 1)             # Apenas famílias de uma pessoa
df_filtrado = df[filtro].copy()
df_filtrado

,e401k,inc,male,age,fsize,nettfa
0,0,13.170,0,40,1,4.575
1,1,61.230,1,35,1,154.000
4,0,22.614,0,53,1,18.450
9,1,29.100,1,45,1,29.600
15,0,19.074,1,43,1,0.000
...,...,...,...,...,...,...
9258,0,28.572,1,28,1,-0.180
9259,1,57.282,0,36,1,28.579
9260,0,73.458,0,30,1,9.900
9267,0,16.968,0,49,1,69.200


### Solução sugerida

In [5]:
# Regressão pelo Método de Mínimos Quadrados Ponderados (MQP ou WLS na sigla em inglês)

'''
Hipótese utilizada nesta tabela para execução da regressão pelo método de Mínimos Quadrados Ponderados:
var(u|inc) = sigma_quad * inc    -->  peso = 1 / inc
'''
peso_mqp = 1 / df_filtrado['inc'] 

modelo = smf.wls('nettfa ~ inc + np.square(age - 25) + male + e401k', weights=peso_mqp, data=df_filtrado)
reg = modelo.fit()

In [6]:
tabela = pd.DataFrame({'estimativas MQP': round(reg.params, 3), 
                       'nonrobust SE': round(reg.bse, 3), 
                       'robust SE': round(reg.HC0_se, 3)},
                      index=['inc', 'np.square(age - 25)', 'male', 'e401k', 'Intercept'])

print('Tabela 8.2 [Variável dependente: nettfa]\n')
print(tabela)
print(f'\nObservações: {int(reg.nobs)}')
print(f'R-quadrado: {round(reg.rsquared, 3)}')

Tabela 8.2 [Variável dependente: nettfa]

                     estimativas MQP  nonrobust SE  robust SE
inc                            0.740         0.064      0.075
np.square(age - 25)            0.018         0.002      0.003
male                           1.841         1.564      1.309
e401k                          5.188         1.703      1.570
Intercept                    -16.703         1.958      2.240

Observações: 2017
R-quadrado: 0.112
